In [283]:
import scipy.io
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [284]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [285]:
df = pd.read_csv('/content/drive/MyDrive/all_faults.csv')

feature extraction

In [286]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

win_len=1028
stride=500

X=[]
Y=[]


for k in df['fault'].unique():

    df_temp_2 = df[df['fault']==k]

    for i in np.arange(0,len(df_temp_2)-(win_len),stride):
        temp = df_temp_2.iloc[i:i+win_len,:-1].values
        temp = temp.reshape((1,-1))
        X.append(temp)
        Y.append(df_temp_2.iloc[i+win_len,-1])

X=np.array(X)
X=X.reshape((X.shape[0],-1,1))
#X = np.repeat(X, 3, axis=3) # To repeat into 3 chanel format


Y=np.array(Y)
encoder= LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
OHE_Y = to_categorical(encoded_Y)

In [287]:
X.shape

(5544, 1028, 1)

Data split


In [288]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,OHE_Y,test_size=0.3,shuffle=True)


CNN　Model

In [289]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input,Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Conv1D,MaxPooling1D



# from tensorflow.keras.models import load_model
# cnn_model = load_model(r'H:\Gear Fault\Trained Model\CNN_model_gear.h5')



no_classes = len(df['fault'].unique())

cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=100, activation='relu', input_shape=(X.shape[1],X.shape[2])))
cnn_model.add(Conv1D(filters=32, kernel_size=50, activation='relu'))
cnn_model.add(Dropout(0.5))



cnn_model.add(Flatten())
cnn_model.add(Dense(100, activation='relu'))


cnn_model.add(Dense(no_classes, activation='softmax'))

cnn_model.summary()

cnn_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_49 (Conv1D)              │ (None, 929, 64)        │         6,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_50 (Conv1D)              │ (None, 880, 32)        │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 880, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_24 (Flatten)            │ (None, 28160)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 100)            │     2,816,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 14)             │         1,414 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,926,410 (11.16 MB)

 Trainable params: 2,926,410 (11.16 MB)

 Non-trainable params: 0 (0.00 B)

train cnn

In [ ]:
batch_size =300
epochs = 10
history = cnn_model.fit(X_train, y_train, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test,y_test),shuffle=True)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 94s 7s/step - accuracy: 0.2334 - loss: 2.2094 - val_accuracy: 0.4982 - val_loss: 1.4186
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 145s 7s/step - accuracy: 0.5614 - loss: 1.2118 - val_accuracy: 0.7338 - val_loss: 0.8125
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 93s 7s/step - accuracy: 0.7854 - loss: 0.7106 - val_accuracy: 0.8900 - val_loss: 0.4525
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 145s 8s/step - accuracy: 0.9167 - loss: 0.3114 - val_accuracy: 0.9381 - val_loss: 0.2068
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 160s 9s/step - accuracy: 0.9557 - loss: 0.1397 - val_accuracy: 0.9393 - val_loss: 0.1401
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 125s 8s/step - accuracy: 0.9673 - loss: 0.1069 - val_accuracy: 0.9327 - val_loss: 0.2266
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 0.9352 - loss: 0.2396 - val_accuracy: 0.9165 - val_loss: 0.2488
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9574 - loss: 0.1393

accuracy graph

In [ ]:
plt.plot(history.history['accuracy'],'go--')
plt.plot(history.history['val_accuracy'],'ro--')
plt.legend(['accuray','val_accuracy'])
plt.grid(color='k', linestyle='--', linewidth=0.5)
plt.show()

confusion matrix

In [ ]:
def inv_Transform_result(y_pred):
    y_pred = y_pred.argmax(axis=1)
    y_pred = encoder.inverse_transform(y_pred)
    return y_pred



y_pred=cnn_model.predict(X_test)


Y_pred=inv_Transform_result(y_pred)
Y_test = inv_Transform_result(y_test)




from sklearn.metrics import confusion_matrix


plt.figure(figsize=(10,10))
cm = confusion_matrix(Y_test, Y_pred,normalize='true')
f = sns.heatmap(cm, annot=True,xticklabels=encoder.classes_,yticklabels=encoder.classes_)
plt.show()

In [ ]:
dummy_cnn = Model(inputs=cnn_model.layers[0].input,outputs=cnn_model.layers[4].output)
y_viz = dummy_cnn.predict(X_train)

T-SNE plot after CNN

In [ ]:
from sklearn.manifold import TSNE

X_t_sne = TSNE(n_components=2, learning_rate='auto',verbose=1, perplexity=40, n_iter=300).fit_transform(y_viz)

tSNEdf = pd.DataFrame(data = X_t_sne, columns = ['T-SNE component 1', 'T-SNE component 2'])

tSNEdf['Fault']=inv_Transform_result(y_train)


# Plot the PC-1 and PC-2
fig, ax = plt.subplots(figsize=(10,10))
sns.scatterplot(x=tSNEdf['T-SNE component 1'],y=tSNEdf['T-SNE component 2'],hue='Fault',
    data=tSNEdf,
    legend="full",
    alpha=0.7)
plt.show()